In [279]:
import os
import sys
import re
#update with the path to your config file
sys.path.append("D:/DataBaseUpdate/To_clone")
from ConfigFiles import config

from lxml import etree
import csv
import unittest
from collections import defaultdict

## Clean Code

#### Helper Functions

In [280]:
def get_xml(valid_xml_file):
    '''
    Return a xml object with patent data
    :param valid_xml_file: a valid xml object
    :return the root object of the xml file, ready to parse
    '''
    tree = etree.parse(valid_xml_file)
    root = tree.getroot()
    return root

In [281]:
def process_patent_numbers(raw_patent_num):
    '''
    Helper function ot transform patent ids into thier final format
    :param raw_patent_num: patent number extracted from the raw XML
    :return cleaned patent id.
    '''
    num = re.findall('\d+', raw_patent_num)[0] #get the just numbers in string form
    if num[0].startswith("0"):
        num = num[1:]
        let = re.findall('[a-zA-Z]+', raw_patent_num) #get the letter prefixes
    if let:
        let = let[0]#list to string
        clean_patent_num = let + num
    else:
        clean_patent_num = num
    return clean_patent_num
    

In [282]:
def process_date(date):
    '''
    Takes a date formated as 6 numbers and returns it with dashes and days that are 00 replaced with 01
    :params date: a date object formatted as 6 numbers
    :returns cleaned up date
    '''
    if date[6:] != "00":
        date = date[:4]+'-'+date[4:6]+'-'+date[6:]
    else:
        date = date[:4]+'-'+date[4:6]+'-'+'01'
    return date

#### Data Functions

In [175]:
def get_patent(patent):
    '''
    :params patent: take the xml object representing a patent
    :returns a default diction with all the data for the patent table
    '''
    #publication reference information
    patent_data = defaultdict(lambda x : "")
    pub_ref_data = patent.findall('.//publication-reference/document-id/')
    for field in pub_ref_data:
        patent_data[field.tag] = field.text
    patent_data['doc-number'] = process_patent_numbers(patent_data['doc-number'])
    patent_data['date'] = process_date(patent_data['date'])
    #abstract
    abst = patent.findall('abstract/p')
    if abst: #not all patents have abstracts
        #this grabs only the first a paragraph and doesn't overwrite it with none if there is a dangling 'None' paragraph
        patent_data['abstract'] = abst[0].text
    #title
    for patent in patents:
        title_data = patent.find('.//invention-title')
        if title_data is not None:
            patent_data['title'] = title_data.text
    return patent_data

## To Run Process

In [210]:
patents = get_xml("D:/DataBaseUpdate/Development/Data/ipg171212_clean.xml")

## Working Area

In [182]:
patents #is an xml-tree object that has all the patents an information
patents[104] #is the element that represnts the 104th patent 

<Element us-patent-grant at 0x1a67a808>

In [141]:
#The main highest level grouings are things like 'abstract','figures', and 'us-bibliographic-data-grant
#'us-bibliographic-data-grant' has most of the fields we are working with

In [268]:
patent = patents[10]

In [ ]:
for patent in patents:
    get_patent(patent)